## 1. Import Required Libraries
Import all necessary libraries for deep learning, data processing, and model training

In [8]:
import os, random
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['ABSL_LOG_LEVEL'] = 'FATAL'

import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

## 2. Set Random Seeds
Ensure reproducibility of experiments by setting consistent random seeds for all operations

In [9]:
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

## 3. Data Preprocessing and Augmentation
Define image data generators with normalization, rotation, flipping, and other data augmentation techniques

In [10]:
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

def pytorch_normalize(img):
    img = img / 255.0
    return (img - mean) / std

train_datagen = ImageDataGenerator(
    preprocessing_function=pytorch_normalize,
    rotation_range=15,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.1,
    zoom_range=[0.95, 1.05],
    brightness_range=[0.85, 1.15],
    horizontal_flip=True,
    channel_shift_range=0.02,
    fill_mode='reflect'
)

val_test_datagen = ImageDataGenerator(
    preprocessing_function=pytorch_normalize
)

## 4. Create Data Generators
Create data generators for training, validation, and test sets, and display dataset information

In [11]:
base_dir = '../data/garbage-dataset'
train_dir = '../data/garbage-split/train'
val_dir = '../data/garbage-split/val'
test_dir = '../data/garbage-split/test'

train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=(224, 224), batch_size=32, class_mode='categorical', shuffle=True, seed=SEED
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir, target_size=(224, 224), batch_size=32, class_mode='categorical', shuffle=False
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir, target_size=(224, 224), batch_size=32, class_mode='categorical',shuffle=False
)

train_steps = int(np.ceil(train_generator.samples / 32))
val_steps = int(np.ceil(val_generator.samples / 32))

Found 15806 images belonging to 10 classes.
Found 2963 images belonging to 10 classes.
Found 993 images belonging to 10 classes.


## 5. Build MobileNetV2 Model
Use pre-trained MobileNetV2 as base model and add custom classification layers

In [12]:
base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model.trainable = True
for layer in base_model.layers[:-30]:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(len(train_generator.class_indices), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

## 6. Model Training
Train the model using callbacks including early stopping, learning rate reduction, and model checkpointing

In [13]:
checkpoint = ModelCheckpoint("saved_models/best_mobilenetv2.keras",  monitor='val_accuracy', mode='max', save_best_only=True, verbose=1)
early_stop = EarlyStopping(monitor='val_accuracy', patience=8, restore_best_weights=True, min_delta=0.001, verbose=1)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-6, verbose=1)

history = model.fit(
    train_generator,
    steps_per_epoch=train_steps,
    validation_data=val_generator,
    validation_steps=val_steps,
    epochs=30,
    callbacks=[checkpoint, early_stop, lr_reduce],
    verbose=1
)

model.save("saved_models/mobilenetv2_final.keras")

/home/eric-u/miniconda3/envs/tf216/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
494/494 ━━━━━━━━━━━━━━━━━━━━ 0s 346ms/step - accuracy: 0.6587 - loss: 1.0668
Epoch 1: val_accuracy improved from -inf to 0.88019, saving model to saved_models/best_mobilenetv2.keras
494/494 ━━━━━━━━━━━━━━━━━━━━ 207s 378ms/step - accuracy: 0.6590 - loss: 1.0660 - val_accuracy: 0.8802 - val_loss: 0.3676 - learning_rate: 1.0000e-04
Epoch 2/30
494/494 ━━━━━━━━━━━━━━━━━━━━ 0s 322ms/step - accuracy: 0.8858 - loss: 0.3535
Epoch 2: val_accuracy improved from 0.88019 to 0.92204, saving model to saved_models/best_mobilenetv2.keras
494/494 ━━━━━━━━━━━━━━━━━━━━ 167s 338ms/step - accuracy: 0.8858 - loss: 0.3534 - val_accuracy: 0.9220 - val_loss: 0.2598 - learning_rate: 1.0000e-04
Epoch 3/30
494/494 ━━━━━━━━━━━━━━━━━━━━ 0s 350ms/step - accuracy: 0.9224 - loss: 0.2384
Epoch 3: val_accuracy improved from 0.92204 to 0.92744, saving model to saved_models/best_mobilenetv2.keras
494/494 ━━━━━━━━━━━━━━━━━━━━ 181s 367ms/step - accuracy: 0.9224 - loss: 0.2384 - val_accuracy: 0.9274 - val_loss: 0.2

In [14]:
test_generator.reset()
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)
y_true = test_generator.classes

print(classification_report(y_true, y_pred, target_names=list(test_generator.class_indices.keys())))

32/32 ━━━━━━━━━━━━━━━━━━━━ 8s 176ms/step
              precision    recall  f1-score   support

     battery       1.00      1.00      1.00        48
  biological       1.00      0.92      0.96        50
   cardboard       0.99      0.92      0.96        92
     clothes       0.99      0.99      0.99       267
       glass       0.99      0.95      0.97       154
       metal       0.94      0.96      0.95        51
       paper       0.87      0.98      0.92        84
     plastic       0.93      0.98      0.96       100
       shoes       0.93      0.97      0.95        99
       trash       0.93      0.90      0.91        48

    accuracy                           0.96       993
   macro avg       0.96      0.96      0.96       993
weighted avg       0.96      0.96      0.96       993

